# System Init

In [1]:
from fov import FOV
from useq import MDAEvent
import pandas as pd
import random
import os
import napari
import pymmcore_plus
from napari_micromanager import MainWindow

from utils import create_folders
from useq._mda_event import SLMImage

mmc = pymmcore_plus.CMMCorePlus()
mmc.loadSystemConfiguration("E:\\MicroManagerConfigs\\Ti2CicercoConfig_w_DMD_21_w_ttl.cfg")
mmc.mda.engine.use_hardware_sequencing = False

### Init for Niesen

In [2]:
import requests
import threading
import time
class WakeUpLaser:
    def __init__(self, lumencore_ip="192.168.201.200"): 
        self.ip = lumencore_ip
        self.last_wakeup = 0
        self.is_running = False
        
    def wakeup_laser(self):
        url = f"http://{self.ip}/service/?command=WAKEUP"
        requests.get(url)
    
    def run(self, wait_for_warmup=False):
        self.is_running = True
        self.thread = threading.Thread(target=self._keep_alive)
        self.thread.start()
        if wait_for_warmup:
            time.sleep(15)

    def _keep_alive(self):
        while self.is_running:
            if time.time() - self.last_wakeup > 60:
                self.wakeup_laser()
                self.last_wakeup = time.time()
            time.sleep(3)
    def stop(self):
        self.is_running = False
        self.thread.join()
        
wl = WakeUpLaser()
wl.wakeup_laser()


slm_dev = mmc.getSLMDevice()
slm_width = mmc.getSLMWidth(slm_dev)
slm_height = mmc.getSLMHeight(slm_dev)

event_slm_on = MDAEvent(slm_image=SLMImage(data=True))
mmc.mda.run([event_slm_on])# to only have fov of DMD 
mmc.setROI(150, 150, 1900, 1900)

DMD_CHANNEL_GROUP = "WF_DMD"
DMD_CALIBRATION_PROFILE = {"channel_group": "WF_DMD", "channel_config": "CyanStim", "device_name": "LedDMD", "property_name": "Cyan_Level", "power": 100}
mmc.setChannelGroup(channelGroup=DMD_CHANNEL_GROUP)

[01/16/25 22:49:28] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=262729;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=502809;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#328\328]8;;\

                    INFO     slm_image=SLMImage()                                                    ]8;id=462729;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=282474;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\

                    INFO     MDA Finished: GeneratorMDASequence()                                    ]8;id=872149;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=686312;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#415\415]8;;\

### Napari Micromanger User Interface

In [3]:
viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)

In [ ]:
from pymmcore_widgets.mda import MDAWidget
mdawidget = MDAWidget(mmcore = mmc)
viewer.window.add_dock_widget(mdawidget)

In [45]:
mm_wdg._core_link.cleanup()

In [15]:
from napari_micromanager._core_link import CoreViewerLink
mm_wdg._core_link = CoreViewerLink(viewer, mmc)

## create a DF with all planned acquisitions and stimulations

In [6]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

path = "C:\\test\\"
create_folders(path,['stim','raw','labels','stim_mask','tracks','labels_rings','particles'])

time_between_frames = 20 #time in seconds between frames

timesteps = range(3)  
channels = ['Red', 'Green'] #channel for segmentation first
channels_exposure = [500, 500]

# take values from UI, if loaded
intensity_red_laser = mmc.getProperty("Laser", "RED_Intensity")
intensity_green_laser = mmc.getProperty("Laser", "GREEN_Intensity")

if intensity_red_laser != str(0) and intensity_green_laser != str(0):
    channels_power = [intensity_red_laser, intensity_green_laser]
else:
    channels_power = [48, 33]
    for channel, power in zip(channels, channels_power):
        mmc.setProperty("Laser", f"{channel.upper()}_Intensity", power)

cell_lines = ["optoFGFR1", "optoERK1", "optoERK1"]

stim_exposures = [100] # list of possible exposures
stim_timesteps = [[0]]  # list of timesteps
stim_profiles = [{"device_name": "LedDMD", "property_name": "Cyan_Level", "power": 10, "channel": "CyanStim"}]
stim_treatment = [{"stim_property": "global", "stim_profile": stim_profile, "stim_exposure": stim_exposure, "stim_timestep": stim_timestep} 
                  for stim_profile in stim_profiles for stim_exposure in stim_exposures for stim_timestep in stim_timesteps]
random.shuffle(stim_treatment)


data_mda_fovs = None

Directory C:\test\stim already exists
Directory C:\test\raw already exists
Directory C:\test\labels already exists
Directory C:\test\stim_mask already exists
Directory C:\test\tracks already exists
Directory C:\test\labels_rings already exists
Directory C:\test\particles already exists


### Load points from file

In [8]:
import json
file = os.path.join(path, "test.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)


### Using autogenerated points

In [ ]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value()
n_fovs = len(data_mda_fovs.stage_positions)
pts_per_well = data_mda_fovs.stage_positions.well_points_plan.num_points
n_wells = n_fovs // pts_per_well

for i, row in enumerate(data_mda_fovs.stage_positions):
    well_column = int(row.name.split('_')[0][1:])
    well_row = row.name.split('_')[0][0]
    well_id = i%pts_per_well
    cell_line = cell_lines[well_column-1]
    fov = FOV(pos=(row.x, row.y),
              index=i,
              name=row.name,
              path=path,
              metadata={"well_column": well_column, "well_row": well_row, "well_id": well_id, "cell_line": cell_line},
              treatment=stim_treatment[i%len(stim_treatment)],
              )
    fovs.append(fov)

### Manually defined points

In [9]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value().stage_positions
n_fovs = len(data_mda_fovs)
n_fovs_per_stim_condition = 2
stim_treatment_tot = stim_treatment * n_fovs_per_stim_condition
random.shuffle(stim_treatment_tot)

for i, row in enumerate(data_mda_fovs):
    row = dict(row)
    fov = FOV(pos=(row["x"], row["y"]),
              index=i,
              name=str(i),
              path=path,
              metadata={},
              treatment=stim_treatment_tot[i],
              )
    fovs.append(fov)

### Generate timetable

In [10]:
time_per_fov = 10
n_fovs_simultaneously = time_between_frames // time_per_fov
start_time = 0

dfs = []
for fov in fovs:
    fov_group = fov.index // n_fovs_simultaneously
    start_time = fov_group * time_between_frames * len(timesteps)

    for timestep in timesteps:
        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'name':fov.name,
                    'timestep': timestep,
                    'time': start_time + timestep*time_between_frames,
                    'treatment': fov.treatment,
                    'metadata': fov.metadata,
                    'stim': timestep in fov.treatment['stim_timestep'],
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_power': channels_power,
                    'stim_profile' : fov.treatment['stim_profile'],
                    'stim_exposure' : fov.treatment['stim_exposure'],
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    }
        dfs.append(new_row)

df_acquire = pd.DataFrame(dfs)
df_acquire = df_acquire.sort_values(by=['time', 'fov'])
print(f"Total Experiment Time: {df_acquire['time'].max()}s")
df_acquire


Total Experiment Time: 40s


,fov_object,fov,name,timestep,time,treatment,metadata,stim,channels,channels_exposure,channel_power,stim_profile,stim_exposure,fname
0,<fov.FOV object at 0x0000023856A4CC10>,0,0,0,0,"{'stim_property': 'global', 'stim_profile': {'...",{},True,"[Red, Green]","[500, 500]","[48, 33]","{'device_name': 'LedDMD', 'property_name': 'Cy...",100,000_00000
1,<fov.FOV object at 0x0000023856A4CC10>,0,0,1,20,"{'stim_property': 'global', 'stim_profile': {'...",{},False,"[Red, Green]","[500, 500]","[48, 33]","{'device_name': 'LedDMD', 'property_name': 'Cy...",100,000_00001
2,<fov.FOV object at 0x0000023856A4CC10>,0,0,2,40,"{'stim_property': 'global', 'stim_profile': {'...",{},False,"[Red, Green]","[500, 500]","[48, 33]","{'device_name': 'LedDMD', 'property_name': 'Cy...",100,000_00002


## Run on system

In [11]:
%load_ext autoreload
%autoreload 2

from add_frame import ImageProcessingPipeline
from segmentation_stardist import SegmentatorStardist
from stimulation import StimWholeFOV    
from controller import Controller, Analyzer
from tracking_trackpy import TrackerTrackpy
from dmd import DMD
from queue import Queue

segmentator = SegmentatorStardist()
stimulator = StimWholeFOV()
tracker = TrackerTrackpy()
dmd = DMD(mmc, DMD_CALIBRATION_PROFILE)

pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)
queue = Queue()
controller = Controller(analyzer, mmc, queue, dmd)
wl.run(wait_for_warmup=True)
controller.run(df_acquire)
wl.stop()
print("finished")

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
index={'t': 0, 'c': 0} channel=Channel(config='Red', group='WF_DMD') exposure=500.0 min_start_time=0.0 x_pos=-3047.1 y_pos=-3719.6 metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov': 0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment': {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim': True, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power': ['48', '33'], 'stim_profile': {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00000', 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}
index=

[01/16/25 22:51:00] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=999866;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=99088;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#328\328]8;;\

                    INFO     index={'t': 0, 'c': 0} channel=Channel(config='Red', group='WF_DMD')    ]8;id=739031;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=666551;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             exposure=500.0 min_start_time=0.0 x_pos=-3047.1 y_pos=-3719.6                         
                             metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov':                
                             0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment':                                
                             {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD',                 
                             'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'},                   
                             'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim':                  
                             True, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500],                  
                             'channel_power': ['48', '33'], 'stim_profile': {'device_name':                        
                             'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel':                      
                             'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00000', 'img_type':                  
                             <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}                        

New frame ready


[01/16/25 22:51:01] INFO     index={'t': 0, 'c': 1} channel=Channel(config='Green', group='WF_DMD')  ]8;id=453383;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=487182;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             exposure=500.0 min_start_time=0.0 x_pos=-3047.1 y_pos=-3719.6                         
                             metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov':                
                             0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment':                                
                             {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD',                 
                             'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'},                   
                             'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim':                  
                             True, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500],                  
                             'channel_power': ['48', '33'], 'stim_profile': {'device_name':                        
                             'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel':                      
                             'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00000', 'img_type':                  
                             <ImgType.IMG_RAW: 1>, 'last_channel': True, 'channel': 'Green'}                       

...

New frame ready
Store stim image
index={'t': 1, 'c': 0} channel=Channel(config='Red', group='WF_DMD') exposure=500.0 min_start_time=20.0 x_pos=-3047.1 y_pos=-3719.6 metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov': 0, 'name': '0', 'timestep': 1, 'time': 20, 'treatment': {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim': False, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power': ['48', '33'], 'stim_profile': {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00001', 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}
index={'t': 1, 'c': 1} channel=Channel(config='Green', group='WF_DMD') exposure=500.0 min_start_time=20.0 x_pos=-3047.1 y_pos=-3719.6 metadata={'fov_object': <fov.FOV obje

[01/16/25 22:51:20] INFO     index={'t': 1, 'c': 0} channel=Channel(config='Red', group='WF_DMD')    ]8;id=736143;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=334002;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             exposure=500.0 min_start_time=20.0 x_pos=-3047.1 y_pos=-3719.6                        
                             metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov':                
                             0, 'name': '0', 'timestep': 1, 'time': 20, 'treatment':                               
                             {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD',                 
                             'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'},                   
                             'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim':                  
                             False, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500],                 
                             'channel_power': ['48', '33'], 'stim_profile': {'device_name':                        
                             'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel':                      
                             'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00001', 'img_type':                  
                             <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}                        

New frame ready


[01/16/25 22:51:21] INFO     index={'t': 1, 'c': 1} channel=Channel(config='Green', group='WF_DMD')  ]8;id=954235;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=90616;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#289\289]8;;\
                             exposure=500.0 min_start_time=20.0 x_pos=-3047.1 y_pos=-3719.6                        
                             metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov':                
                             0, 'name': '0', 'timestep': 1, 'time': 20, 'treatment':                               
                             {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD',                 
                             'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'},                   
                             'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim':                  
                             False, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500],                 
                             'channel_power': ['48', '33'], 'stim_profile': {'device_name':                        
                             'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel':                      
                             'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00001', 'img_type':                  
                             <ImgType.IMG_RAW: 1>, 'last_channel': True, 'channel': 'Green'}                       

New frame ready
Segmenting image
Store raw image
index={'t': 2, 'c': 0} channel=Channel(config='Red', group='WF_DMD') exposure=500.0 min_start_time=40.0 x_pos=-3047.1 y_pos=-3719.6 metadata={'fov_object': <fov.FOV object at 0x0000023856A4CC10>, 'fov': 0, 'name': '0', 'timestep': 2, 'time': 40, 'treatment': {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {}, 'stim': False, 'channels': ['Red', 'Green'], 'channels_exposure': [500, 500], 'channel_power': ['48', '33'], 'stim_profile': {'device_name': 'LedDMD', 'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure': 100, 'fname': '000_00002', 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'Red'}
index={'t': 2, 'c': 1} channel=Channel(config='Green', group='WF_DMD') exposure=500.0 min_start_time=40.0 x_pos=-3047.1 y_pos=-3719.6 metadata={'fov_object

In [28]:
df_acquire["fov_object"].loc[4].tracks

,mean_intensity_C0_nuc,mean_intensity_C1_nuc,label,x,y,mean_intensity_C0_ring,mean_intensity_C1_ring,particle,frame,stim,...,acquired,channels,channels_exposure,channel_power,stim_profile,stim_exposure,fname,img_type,last_channel,channel
0,15794.850310,7030.674491,1,1449.204606,1157.172276,2027.836054,8764.606813,0,0,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
1,10468.702200,8982.124083,2,1037.380147,1067.925550,1929.307385,11140.400532,1,0,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
2,6090.786316,5357.677368,3,1807.577368,779.328947,1588.041935,5225.286452,2,0,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
3,11482.417821,7621.467790,4,1030.042745,301.026490,2115.942584,7030.071087,3,0,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
4,13195.260595,6885.135257,5,1257.035167,47.868350,3160.891010,6125.711217,4,0,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,1678.256503,4662.350434,51,1552.354769,1304.611994,1349.972898,4863.286504,51,3,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
203,2738.794118,5727.941176,52,973.235294,390.985294,1649.526316,5775.962963,49,3,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
204,1799.209811,4791.371321,53,65.895094,1239.891321,1355.067164,4690.520149,53,3,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green
205,2103.364269,5779.733643,54,1159.126682,685.616241,1748.901220,5837.039634,48,3,False,...,False,"[Red, Green]","[500, 500]","[48, 33]",NaN,100.0,000_00003,ImgType.IMG_RAW,True,Green


In [27]:
import pandas as pd
pd.read_parquet(os.path.join(path, "tracks", "000_00004.parquet")).columns

Index(['mean_intensity_C0_nuc', 'mean_intensity_C1_nuc', 'label', 'x', 'y',
       'mean_intensity_C0_ring', 'mean_intensity_C1_ring', 'particle', 'frame',
       'stim', 'time', 'fov', 'name', 'timestep', 'treatment', 'metadata',
       'acquired', 'channels', 'channels_exposure', 'channel_power',
       'stim_profile', 'stim_exposure', 'fname', 'last_channel'],
      dtype='object')